In [2]:
import os
import time
import os.path as osp

import numpy as np
import pandas as pd

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader

from torchvision import datasets
from torchvision import transforms

import torchvision

import matplotlib.pyplot as plt
from PIL import Image
from sklearn.metrics import recall_score, f1_score
import matplotlib.pyplot as plt
from sklearn.metrics import roc_auc_score, roc_curve

In [6]:
# random seed
SEED = 1 
NUM_CLASS = 10

# Training
BATCH_SIZE = 128
NUM_EPOCHS = 30
EVAL_INTERVAL=1
SAVE_DIR = './log'

# Optimizer
LEARNING_RATE = 1e-1
MOMENTUM = 0.9
STEP=5
GAMMA=0.5
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
transform_cifar100_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])
#裁剪到相同scale

transform_cifar100_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

train_set = torchvision.datasets.CIFAR10(root='../data', train=True,
                                        download=True, transform=transform_cifar100_train)
train_dataloader = torch.utils.data.DataLoader(train_set, batch_size=BATCH_SIZE,
                                          shuffle=True, num_workers=2)

test_set = torchvision.datasets.CIFAR10(root='../data', train=False,
                                       download=True, transform=transform_cifar100_test)
test_dataloader = torch.utils.data.DataLoader(test_set, batch_size=BATCH_SIZE,
                                         shuffle=False, num_workers=2)

class_names = ['apple', 'aquarium_fish', 'baby', 'bear', 'beaver', 'bed', 'bee', 'beetle', 'bicycle', 'bottle', 'bowl', 'boy', 'bridge', 'bus', 'butterfly', 'camel', 'can', 'castle', 'caterpillar', 'cattle', 'chair', 'chimpanzee', 'clock', 'cloud', 'cockroach', 'couch', 'crab', 'crocodile', 'cup', 'dinosaur', 'dolphin', 'elephant', 'flatfish', 'forest', 'fox', 'girl', 'hamster', 'house', 'kangaroo', 'keyboard', 'lamp', 'lawn_mower', 'leopard', 'lion', 'lizard', 'lobster', 'man', 'maple_tree', 'motorcycle', 'mountain', 'mouse', 'mushroom', 'oak_tree', 'orange', 'orchid', 'otter', 'palm_tree', 'pear', 'pickup_truck', 'pine_tree', 'plain', 'plate', 'poppy', 'porcupine', 'possum', 'rabbit', 'raccoon', 'ray', 'road', 'rocket', 'rose', 'sea', 'seal', 'shark', 'shrew', 'skunk', 'skyscraper', 'snail', 'snake', 'spider', 'squirrel', 'streetcar', 'sunflower', 'sweet_pepper', 'table', 'tank', 'telephone', 'television', 'tiger', 'tractor', 'train', 'trout', 'tulip', 'turtle', 'wardrobe', 'whale', 'willow_tree', 'wolf', 'woman', 'worm']

class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()
        self.conv1 = nn.Conv2d(3, 4, 3)  
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(4, 8, 3)  
        self.fc1 = nn.Linear(8 * 6 * 6, 32)
        self.fc2 = nn.Linear(32, 10)

    def forward(self, x):
        x = self.pool(torch.relu(self.conv1(x)))
        x = self.pool(torch.relu(self.conv2(x)))
        x = x.view(-1, 8 * 6 * 6)
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x
model = ConvNet()
model=model.cuda()
optimizer = optim.SGD(model.parameters(), lr=LEARNING_RATE, momentum=MOMENTUM)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=STEP, gamma=GAMMA)
criterion =nn.CrossEntropyLoss()
def train_batch(model, image, target):
    output =  model(image)
    loss = criterion(output, target)
    return output, loss

def test_batch(model, image, target):
    output =  model(image)
    loss = criterion(output, target)
    return output, loss


Files already downloaded and verified
Files already downloaded and verified


In [7]:
CRtraining_loss = []
CRtraining_acc = []
CRtesting_loss = []
CRtesting_acc = []
CRtrain_preds = []
CRtrain_targets = []
CRtest_preds = []
CRtest_targets = []
CRtrain_f1_scores = []
CRtest_f1_scores = []
CRlayer_gradients = [[] for _ in range(len(list(model.parameters())))]

for epoch in range(NUM_EPOCHS):
    model.train()
    torch.cuda.empty_cache()

    ##########################
    ### Training
    ##########################

    running_cls_loss = 0.0
    running_cls_corrects = 0

    for batch_idx, (image, target) in enumerate(train_dataloader):
        image = image.to(device)
        target = target.to(device)

        # train model
        outputs, loss = train_batch(model, image, target)
        _, preds = torch.max(outputs, 1)

        CRtrain_preds.extend(preds.cpu().tolist())
        CRtrain_targets.extend(target.cpu().tolist())

        loss_data = loss.data.item()
        if np.isnan(loss_data):
            raise ValueError('loss is nan while training')
        running_cls_loss += loss.item()
        running_cls_corrects += torch.sum(preds == target.data)

        loss.backward()
        for i, (name, param) in enumerate(model.named_parameters()):
            if param.grad is not None:
                CRlayer_gradients[i].append(param.grad.abs().mean().item())

        optimizer.step()
        optimizer.zero_grad()

    CRtrain_recall = recall_score(CRtrain_targets, CRtrain_preds, average='weighted')
    CRtrain_f1 = f1_score(CRtrain_targets, CRtrain_preds, average='weighted')
    CRtrain_f1_scores.append(CRtrain_f1)
    CRepoch_loss = running_cls_loss / len(train_set)
    CRepoch_acc = running_cls_corrects.double() / len(train_set)

    print(f'Epoch: {epoch+1}/{NUM_EPOCHS} Train Loss: {CRepoch_loss:.4f} Acc: {CRepoch_acc:.4f}')
    print(f'Train Recall: {CRtrain_recall:.4f} F1 Score: {CRtrain_f1:.4f}')
    CRtraining_loss.append(CRepoch_loss)
    CRtraining_acc.append(CRepoch_acc.cpu().detach().numpy())
    scheduler.step()
    if (epoch + 1) % EVAL_INTERVAL == 0 or (epoch + 1) == NUM_EPOCHS:
        print('Begin test......')
        model.eval()

        CRval_loss = 0.0
        CRval_corrects = 0
        CRtest_preds = []
        CRtest_targets = []

        for batch_idx, (image, target) in enumerate(test_dataloader):
            image = image.to(device)
            target = target.to(device)

            # test model
            outputs, loss = test_batch(model, image, target)
            _, preds = torch.max(outputs, 1)
            CRtest_preds.extend(preds.cpu().tolist())
            CRtest_targets.extend(target.cpu().tolist())

            CRval_loss += loss.item()
            CRval_corrects += torch.sum(preds == target.data)

        CRtest_recall = recall_score(CRtest_targets, CRtest_preds, average='weighted')
        CRtest_f1 = f1_score(CRtest_targets, CRtest_preds, average='weighted')
        CRtest_f1_scores.append(CRtest_f1)
        CRval_loss = CRval_loss / len(test_set)
        CRval_acc = CRval_corrects.double() / len(test_set)
        print(f'Test Loss: {CRval_loss:.4f} Acc: {CRval_acc:.4f}')
        print(f'Test Recall: {CRtest_recall:.4f} F1 Score: {CRtest_f1:.4f}')
        CRtesting_loss.append(CRval_loss)
        CRtesting_acc.append(CRval_acc.cpu().detach().numpy())

Epoch: 1/30 Train Loss: 0.0154 Acc: 0.2563
Train Recall: 0.2563 F1 Score: 0.2469
Begin test......
Test Loss: 0.0139 Acc: 0.3557
Test Recall: 0.3557 F1 Score: 0.3344
Epoch: 2/30 Train Loss: 0.0142 Acc: 0.3246
Train Recall: 0.2905 F1 Score: 0.2822
Begin test......
Test Loss: 0.0133 Acc: 0.3769
Test Recall: 0.3769 F1 Score: 0.3507
Epoch: 3/30 Train Loss: 0.0139 Acc: 0.3414
Train Recall: 0.3074 F1 Score: 0.3001
Begin test......
Test Loss: 0.0140 Acc: 0.3450
Test Recall: 0.3450 F1 Score: 0.3229
Epoch: 4/30 Train Loss: 0.0138 Acc: 0.3456
Train Recall: 0.3170 F1 Score: 0.3101
Begin test......
Test Loss: 0.0137 Acc: 0.3426
Test Recall: 0.3426 F1 Score: 0.3142
Epoch: 5/30 Train Loss: 0.0136 Acc: 0.3571
Train Recall: 0.3250 F1 Score: 0.3186
Begin test......
Test Loss: 0.0132 Acc: 0.3701
Test Recall: 0.3701 F1 Score: 0.3491
Epoch: 6/30 Train Loss: 0.0128 Acc: 0.3907
Train Recall: 0.3359 F1 Score: 0.3297
Begin test......
Test Loss: 0.0121 Acc: 0.4363
Test Recall: 0.4363 F1 Score: 0.4305
Epoch: 7/3

In [8]:
import torch
from torchcp.classification.scores import THR, APS, SAPS, RAPS
from torchcp.classification.predictors import SplitPredictor, ClusterPredictor, ClassWisePredictor

weight_for_saps = 1.0
penalty=0.5
score_functions = [THR(), APS(), SAPS(weight=weight_for_saps), RAPS(penalty)]
predictors = [SplitPredictor, ClusterPredictor, ClassWisePredictor]  # 假设您已实现 ClusterPredictor

# 用于存储不同组合的性能结果
performance_results = []

for score_function in score_functions:
    for Predictor in predictors:
        # 创建预测器实例
        predictor = Predictor(score_function=score_function, model=model)

        # 校准数据集的准备
        cal_dataloader = torch.utils.data.DataLoader(train_set, batch_size=BATCH_SIZE, shuffle=True, num_workers=2)

        # 校准预测器
        predictor.calibrate(cal_dataloader, alpha=0.1)

        # 使用测试集进行预测
        predict_sets = []
        for images, _ in test_dataloader:
            images = images.to(device)
            batch_predict_sets = predictor.predict(images)
            predict_sets.extend(batch_predict_sets)

        # 评估覆盖率和平均集合大小
        result_dict = predictor.evaluate(test_dataloader)
        coverage_rate, average_size = result_dict["Coverage_rate"], result_dict["Average_size"]
        
        # 记录性能结果
        performance_results.append((score_function.__class__.__name__, Predictor.__name__, coverage_rate, average_size))

# 打印性能结果
for result in performance_results:
    print(f"Score Function: {result[0]}, Predictor: {result[1]}, Coverage Rate: {result[2]:.4f}, Average Set Size: {result[3]:.4f}")

Score Function: THR, Predictor: SplitPredictor, Coverage Rate: 0.9086, Average Set Size: 3.2963
Score Function: THR, Predictor: ClusterPredictor, Coverage Rate: 0.9110, Average Set Size: 3.3260
Score Function: THR, Predictor: ClassWisePredictor, Coverage Rate: 0.9069, Average Set Size: 3.3107
Score Function: APS, Predictor: SplitPredictor, Coverage Rate: 0.9022, Average Set Size: 3.5150
Score Function: APS, Predictor: ClusterPredictor, Coverage Rate: 0.9017, Average Set Size: 3.4846
Score Function: APS, Predictor: ClassWisePredictor, Coverage Rate: 0.9038, Average Set Size: 3.5068
Score Function: SAPS, Predictor: SplitPredictor, Coverage Rate: 0.9158, Average Set Size: 3.9558
Score Function: SAPS, Predictor: ClusterPredictor, Coverage Rate: 0.9170, Average Set Size: 4.0011
Score Function: SAPS, Predictor: ClassWisePredictor, Coverage Rate: 0.9137, Average Set Size: 3.9660
Score Function: RAPS, Predictor: SplitPredictor, Coverage Rate: 0.9144, Average Set Size: 4.0156
Score Function: RA